[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/odoo_bc_lay_tt.ipynb)

In [0]:
import time
import os
import base64
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from openerp.osv import fields, osv
from openerp.tools.translate import _
from openerp.tools import DEFAULT_SERVER_DATE_FORMAT, DEFAULT_SERVER_DATETIME_FORMAT
from dateutil import parser

class report_online_performance(osv.osv_memory):
    
    _name = "report.online.performance"
    _columns = {
        'date_start': fields.date('Date From', required=True),
        'date_end': fields.date('Date To', required=True),
        'file_output':fields.binary(string="File Output",invisible=True, readonly=True, help='Output file in xlsx format'),
        'file_name':fields.char(string='File Name', invisible=True)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    }
    _defaults = {
         'date_start': time.strftime('%Y-%m-01'),
         'date_end': lambda *a: str(datetime.now() + relativedelta(months=+1, day=1, days=-1))[:10],
    }
    
    out_path = '/tmp/bc_khach_hang_%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    def print_excel(self, cr, uid, ids, context=None):

       
        #set date to query_data
        #============================================================================================#        
        datas = {'ids': context.get('active_ids', [])}
        date_start = self.read(cr, uid, ids, ['date_start'], context=context)
        date_start = date_start[0]['date_start']+' 00:00:00'
        date_end   = self.read(cr, uid, ids, ['date_end'], context=context)
        date_end   = date_end[0]['date_end']+' 23:59:59'
        

        #Run report
        #============================================================================================# 
        def get_data(date_start, date_end):
            
            get_phone =  '''SELECT p.user_id, count(DISTINCT r.phone)::int as sl_phone
                FROM res_partner r INNER JOIN pos_order p ON r.id =  p.partner_id
                WHERE r.phone != ''
                    AND (date(timezone('UTC',r.create_date::timestamp)) BETWEEN '%s' and '%s') 
                    AND (date(timezone('UTC',p.date_order::timestamp)) BETWEEN '%s' and '%s') 
                    AND length(r.phone) between 9 and 11
                    AND r.phone like '0%s'
                group by 1
            '''%(date_start,date_end,date_start,date_end,'%')

            get_email =  '''SELECT p.user_id, count(DISTINCT r.email)::int as sl_email
                        FROM res_partner r INNER JOIN pos_order p ON r.id =  p.partner_id
                        WHERE r.email != ''
                            AND (date(timezone('UTC',r.create_date::timestamp)) BETWEEN '%s' and '%s') 
                            AND r.id = p.partner_id
                            AND r.email like '%s@%s'
                            AND r.email like '%s.com%s'
                            AND r.email not like '%s %s'
                            AND r.email not like '%s,%s'
                            AND (date(timezone('UTC',p.date_order::timestamp)) BETWEEN '%s' and '%s')
                        group by 1
                    '''%(date_start,date_end,'%','%', '%','%', '%','%', '%','%',date_start,date_end)

            get_dob =  '''SELECT foo.user_id, count(DISTINCT foo.phone)::int as sl_dob
                        FROM (
                            SELECT DISTINCT p.pos_reference, p.name, p.user_id, r.phone as phone, DATE_PART('year', now()::date) - DATE_PART('year', r.birthdate_date::date) as tuoi, r.create_uid
                            FROM res_partner r INNER JOIN pos_order p ON r.id =  p.partner_id
                            WHERE r.birthdate_date IS NOT NULL
                                            AND (date(timezone('UTC',r.create_date::timestamp)) BETWEEN '%s' and '%s') 
                                            AND r.id = p.partner_id 
                                            AND (date(timezone('UTC',p.date_order::timestamp)) BETWEEN '%s' and '%s') )foo
                        WHERE foo.tuoi between 18 and 70
                        group by 1
                    '''%(date_start,date_end,date_start,date_end)
            get_order ='''
                SELECT p.user_id, count(DISTINCT p.id)::int as sl_order
                FROM pos_order p
                WHERE (date(timezone('UTC',p.create_date::timestamp)) BETWEEN '%s' and '%s') 
                    AND (date(timezone('UTC',p.date_order::timestamp)) BETWEEN '%s' and '%s')
                    AND p.partner_id IS NULL 
                group by 1
            '''%(date_start,date_end,date_start,date_end)
            get_name = 'select ru.id as user_id, rp.name from res_users ru left join res_partner rp on ru.partner_id = rp.id'
            
            get_phone_df = pd.read_sql(get_phone,engine)
            get_email_df = pd.read_sql(get_email,engine)
            get_dob_df = pd.read_sql(get_dob, engine)
            get_order_df = pd.read_sql(get_order, engine)
            get_name_df = pd.read_sql(get_name, engine)
            
            result = pd.merge(get_name_df, get_phone_df, how='right', on='user_id')
            result = pd.merge(result, get_email_df, how='outer', on='user_id')
            result = pd.merge(result, get_dob_df, how='outer', on='user_id')
            result = pd.merge(result, get_order_df, how='outer', on='user_id').fillna(0)
            result['sl_order'] = result['sl_order'] + result['sl_phone']
            result['phone/order'] = result['sl_phone']/result['sl_order']
            result['email/order'] = result['sl_email']/result['sl_order']
            #result['dob/order'] = result['sl_dob']/result['sl_order']
            result = result.drop(['user_id'], axis=1)
            result = result[['name', 'sl_order', 'sl_phone', 'phone/order', 'sl_email', 'email/order', 'sl_dob']]
            result.columns = ['NVTN', 'order', '#phones', 'Ratio #phone/#order', '#email', 'Ratio #email/#order', '#dob']
            
            writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
            result.to_excel(writer,sheet_name = 'Report', startrow=6, startcol=1, index=False)
            workbook = writer.book
            worksheet = writer.sheets['Report']
            big_size = workbook.add_format({'bold': True, 'font_size':15})
             worksheet.set_column('B:H', 21)
            
            worksheet.write(2,1,'Từ ngày')
            worksheet.write(2,2,date_start)
            worksheet.write(2,6,'Đến ngày')
            worksheet.write(2,7,date_end)
            worksheet.write(0, 2, 'THƯỞNG NVTN THU THẬP THÔNG TIN KHÁCH HÀNG', big_size)
            writer.save()
           
            #Encode final file to binary and download it.
            #============================================================================================#

            PREVIEW_PATH = out_path
            encoded_string = ""     
            with open(PREVIEW_PATH, "r") as image_file:
                encoded_string = base64.encodestring(image_file.read())#.b64encode
            for order in self.browse(cr, uid, ids[0]):
                order.file_name = '%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                order.file_output = encoded_string
                data_id =  order.id

            return {
                'type': 'ir.actions.act_window',
                'res_model': 'report.online.performance',
                'view_mode': 'form',
                'view_type': 'form',
                'res_id': data_id,
                'views': [(False, 'form')],
                'target': 'new',
            }
